# Graph Generic Cyher or QA Graph Cyher are suitable fetching the exact nodes relating to graph

In [1]:
import pinecone
import openai
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
from langchain.embeddings.base import Embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()
neo4j_url = os.getenv('NEO4J_URL')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')
openai_key = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = openai_key
pinecone_api_key = os.getenv('PINECONE_KEY')

embeddings = OpenAIEmbeddings()

/Users/rc/miniforge3/envs/chat/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Step 1. Get names of nodes, edges, labels

In [2]:
# Get all the nodes names

driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))
# Divide the graph into trunks
def get_pair_nodes (tx):
    pairs_of_nodes = []
    for record in tx.run("MATCH (a)-[r]->(b) RETURN labels(a), a.name, type(r), labels(b), b.name"):
        pair_node = {
            "start_node_name": record["a.name"],
            "start_node_label":record["labels(a)"][0],
            "end_node_name": record["b.name"],
            "end_node_label":record["labels(b)"][0],
            "edge": record["type(r)"]
        }
        pairs_of_nodes.append(pair_node)
        # print(record["labels(a)"][0], record["a.name"], record["type(r)"], record["labels(b)"][0], record["b.name"])
    return pairs_of_nodes


In [3]:
with driver.session() as session:
    pairs_of_nodes = session.execute_read(get_pair_nodes)
driver.close()
pairs_of_nodes

[{'start_node_name': 'Review_project_portfolio_balance_for_2020_Q1',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'Portfolio_management',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Conduct_project_assurance_readiness_for_Brexit_Programme',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Identify_best_practice_regulatory_project_as_examplar',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Identify_best_practice_regulatory_project_as_examplar',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Support',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Cross-project_review_of_plastic_dependencies',
  'start_node_label': 'Areas_of_support',


# Step 2: Build vectors

In [4]:
node_names = set()
node_types = set()
edge_types = set()
for p_n in pairs_of_nodes:
    node_names.add(p_n["start_node_name"])
    node_names.add(p_n["end_node_name"])
    
    node_types.add(p_n["start_node_label"])
    node_types.add(p_n["end_node_label"])
    
    edge_types.add(p_n["edge"])

In [5]:
node_vectors = []
for n in node_names:
    ele_info = {
        "value": embeddings.embed_documents([n]),
        "meta_data": {
            "source":"ps-graph-01",
            "info_type": "node_names",
            "name":n
        },
    } 
    node_vectors.append(ele_info)

for n in node_types:
    ele_info = {
        "value": embeddings.embed_documents([n]),
        "meta_data": {
            "source":"ps-graph-01",
            "info_type": "node_types",
            "name":n
        },
    } 
    node_vectors.append(ele_info)

for n in edge_types:
    ele_info = {
        "value": embeddings.embed_documents([n]),
        "meta_data": {
            "source":"ps-graph-01",
            "info_type": "edges",
            "name":n
        },
    } 
    node_vectors.append(ele_info)
node_vectors[:5]

[{'value': [[-0.01019681658917191,
    -0.029660926470466458,
    -0.023608530896450382,
    -0.015347641583025378,
    0.001953399819276949,
    0.024279466882248805,
    -0.02618044907093563,
    0.0018712800986882155,
    -0.021707548707763556,
    0.007617909769803691,
    0.010846784777676194,
    0.0033302147824580005,
    -0.0001513535450978552,
    -0.01829696148235289,
    -0.01179727680334222,
    0.005444359254809624,
    0.035727297471603255,
    -0.018939941025974204,
    0.0001598712761911249,
    -0.004102489145858003,
    -0.011643521027981168,
    0.004598701559001236,
    0.007694787657484216,
    -0.0010867751410817068,
    -0.0008233807499596202,
    -0.0037565379528036805,
    0.01804536095333979,
    -0.03888628416800082,
    0.015557309311418039,
    0.01310420275655637,
    0.01832491606188478,
    -0.010944629531328246,
    -0.005245175425064034,
    -0.0077786546557090195,
    -0.01441112870977052,
    -0.04028406529866144,
    -0.0035678375560438537,
    0.00

In [6]:
# Setting of pinecone
env_name = "us-west4-gcp-free"
index_name = "test-chatbot-ran"
pinecone.init(api_key=pinecone_api_key,environment=env_name)
index = pinecone.Index(index_name)

# upload vectors 
my_namespace = 'graph'
index.delete(deleteAll='true', namespace=my_namespace)

pinecone_vectors = []
for idx, n_v in enumerate(node_vectors):
    pinecone_vectors.append((str(idx), n_v["value"], n_v["meta_data"]))
upsert_response = index.upsert(vectors=pinecone_vectors, namespace=my_namespace)

# Step 3: Add additional hint on top of original question


In [7]:
prompt = "What types of business value does Unilever create?"

In [8]:
env_name = "us-west4-gcp-free"
index_name = "test-chatbot-ran"
pinecone.init(api_key=pinecone_api_key,environment=env_name)
index = pinecone.Index(index_name)

vectorstore = Pinecone(index , embeddings.embed_query, "name", namespace=my_namespace)

In [9]:
docs = vectorstore.similarity_search(prompt, k=10)
docs

[Document(page_content='Value_Unilever_create', metadata={'info_type': 'node_types', 'source': 'ps-graph-01'}),
 Document(page_content='Business_Transformation_themes', metadata={'info_type': 'node_types', 'source': 'ps-graph-01'}),
 Document(page_content='Business_disruption_', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='1._Winning_with_brands_and_innovation', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='Business_transformation_portfolio', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='7._Consumer_insight_', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='Supply_chain_strategy', metadata={'info_type': 'node_types', 'source': 'ps-graph-01'}),
 Document(page_content='4._Marketing', metadata={'info_type': 'node_names', 'source': 'ps-graph-01'}),
 Document(page_content='Divestment_', metadata={'info_type': 'node_names', 'so

In [10]:
related_node_names = ""
related_node_types = ""
related_edges = ""
for d in docs: 
    if d.metadata["info_type"] == "node_names":
        related_node_names+=d.page_content + ", "
    
    if d.metadata["info_type"] == "node_types":
        related_node_types+=d.page_content + ", "
    
    if d.metadata["info_type"] == "edges":
        related_edges+=d.page_content + ", "

print(related_node_names)
print(related_node_types) 
print(related_edges)

Business_disruption_, 1._Winning_with_brands_and_innovation, Business_transformation_portfolio, 7._Consumer_insight_, 4._Marketing, Divestment_, 1._Consumer_benefits, 
Value_Unilever_create, Business_Transformation_themes, Supply_chain_strategy, 



In [11]:
additional_hint = f"""Hint: Please refer the names of nodes:{related_node_names} or the labels of nodes: {related_node_types} or the edges type: {related_edges} if necessary"""
additional_hint

'Hint: Please refer the names of nodes:Business_disruption_, 1._Winning_with_brands_and_innovation, Business_transformation_portfolio, 7._Consumer_insight_, 4._Marketing, Divestment_, 1._Consumer_benefits,  or the labels of nodes: Value_Unilever_create, Business_Transformation_themes, Supply_chain_strategy,  or the edges type:  if necessary'

# Add additional hint on top of QA graph

In [12]:
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.chat_models import ChatOpenAI

graph = Neo4jGraph(
    url=neo4j_url, 
    username=neo4j_user, 
    password=neo4j_password
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

result = chain.run(prompt + additional_hint)
result



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Value_Unilever_create)-[:enabled_by]->(m:Business_Transformation_themes)-[:comprised_of]->(p:Programme)
RETURN n, m, p
Full Context:
[]

> Finished chain.


"Unilever creates various types of business value through its focus on business disruption, winning with brands and innovation, business transformation portfolio, consumer insight, marketing, divestment, and consumer benefits. These efforts contribute to the overall value that Unilever creates as a company. Additionally, Unilever's business transformation themes and supply chain strategy play a role in generating business value."